# Ireland Met Eireann West Coast Historical Wind Data Analysis <br/> Guillermo F. Martin Gonzalez

# Introduction
This notebooks contains all the analysis carried as part of the project for the 2024 **Programming for Data Analytics (PFDA)** course from the Atlantic Technical University. The project was set up quite open both in terms of the data to be used and the questions to be asked. I have followed the suggestion of downloading the historical wind data from the Met Eireann website (https://www.met.ie/climate/available-data/historical-data). I had never used wind or weather data in the past (I come from a fisheries data background), and found weather data to be quite interesting as weather patterns can be quite 

However, I have taken the assessment project a step further by downloading the data from three different weather stations in the West Coast of Ireland. 

# Data 
Three historical weather data `.csv` files were downloaded from the MetEireann website (above) belonging to the stations:

- hly275
- hly2275
- hly2375

The format of these `.csv` files was slightly different in the number of preliminary rows which described the column headers and the units of each column. In each of these files, we manually deleted this columns, and left the header as the first row. This was done to facilitate the loading of the data into Python. The data contained in this `.csv` files is decribed in the `KeyHourly.txt` file, but as a summary: 

- rain:Precipitation Amount (mm)
- temp: Air Temperature (°C)
- wetb: Wet Bulb Air Temperature (°C)
- dewpt: Dew Point Air Temperature (°C)		                 
- vappr: Vapour Pressure (hpa)
- rhum: Relative Humidity (%)
- msl: Mean Sea Level Pressure (hPa)
- wdsp: Mean Hourly Wind Speed (kt)
- wddir: Predominant Hourly wind Direction (deg)
- ww: Synop Code Present Weather - decode in `.txt`
- w: Synop Code Past Weather - decode `.txt`
- sun: Sunshine duration (hours)
- vis: Visibility (m)
- clht: Cloud Ceiling Height - if none value is 999 (100s feet)
- clamt: Cloud Amount (okta)


Additionally, we also downloaded the Station details from this website which contain useful information such as the location of the station or the opening year. 

In [1]:
# Loading libraries
import pandas as pd
import numpy as np 
import os, json 
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import windrose 

In [2]:
# Loading all .csv files in the data folder
path_to_csv = './data'
csv_files = ['hly2375.csv','hly275.csv','hly2275.csv']
print(csv_files)

# Initialize a list to store all data
data_list = []


['hly2375.csv', 'hly275.csv', 'hly2275.csv']


In [4]:
#All .csv files seem to have the .ind column. The header of the data is ind + 2
for index, d in enumerate(csv_files):
    with open(os.path.join(path_to_csv, d)) as csv_file:
        tmp = pd.read_csv(csv_file)
        #tmp['Location'] = d.split('_')[0]
        tmp['StId'] = d.split('.')[0].replace('hly', '')
        data_list.append(tmp) 

data = pd.concat(data_list, ignore_index=True)        

C:\Users\Owner\AppData\Local\Temp\ipykernel_17576\2939649902.py:4: DtypeWarning: Columns (12,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv(csv_file)
C:\Users\Owner\AppData\Local\Temp\ipykernel_17576\2939649902.py:4: DtypeWarning: Columns (2,4,6,7,8,9,10,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv(csv_file)
C:\Users\Owner\AppData\Local\Temp\ipykernel_17576\2939649902.py:4: DtypeWarning: Columns (2,4,6,7,8,9,10,12,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv(csv_file)


In [5]:
data['StId'].unique()

array(['2375', '275', '2275'], dtype=object)